<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>
<h1 align="right">KSO Tutorials #6: Evaluate machine learning models</h1>
<h3 align="right">Written by the KSO Team</h3>

# Set up KSO requirements

In [ ]:
# @title <font size="5">↓ ឵឵<i>Install kso-object-detection and its requirements</font> { vertical-output: true }
from IPython.display import clear_output

try:
    import google.colab
    import os
    import sys

    IN_COLAB = True
    print("Running in Colab...")

    # Clone repo
    !git clone --recurse-submodules https://github.com/ocean-data-factory-sweden/kso-object-detection.git
    %pip install -q --upgrade pip
    %pip install -qr <(sed '/Pillow/d;/ipywidgets/d' kso-object-detection/requirements.txt)
    %pip install -qr <(sed '/Pillow/d;/ipywidgets/d' kso-object-detection/yolov5_tracker/requirements.txt)
    %pip install -qr <(sed '/Pillow/d;/ipywidgets/d' kso-object-detection/kso_utils/requirements.txt)

    # Fix libmagic issue
    !apt-get -qq update && apt-get -qq install -y libmagic-dev > /dev/null

    # Replace upsampling script with custom version
    os.chdir("kso-object-detection/tutorials")
    sys.path.append("..")

    # Replace nearest neighbours script with custom version (due to relative path issue)
    !cp ../src/multi_tracker_zoo.py ../yolov5_tracker/trackers/strong_sort/multi_tracker_zoo.py

    # Enable external widgets
    from google.colab import output

    output.enable_custom_widget_manager()

    # Ensure widgets are shown properly
    !jupyter nbextension enable --user --py widgetsnbextension
    !jupyter nbextension enable --user --py jupyter_bbox_widget

    print("All packages are installed and ready to go!")
    try:
        clear_output()
        print("All packages are installed and ready to go!")
    except:
        clear_output()
        print("There have been some issues installing the packages!")
except:
    IN_COLAB = False
    import pkgutil

    if (
        pkgutil.find_loader("torch") is None
        or pkgutil.find_loader("torchvision") is None
    ):
        %pip install -q --upgrade pip
        %pip install -q torch torchvision

    # Replace nearest neighbours script with custom version (due to relative path issue)
    !cp ../src/multi_tracker_zoo.py ../yolov5_tracker/trackers/strong_sort/multi_tracker_zoo.py
    # Ensure widgets are shown properly
    !jupyter nbextension enable --user --py widgetsnbextension
    !jupyter nbextension enable --user --py jupyter_bbox_widget
    clear_output()
    print("Running locally... you're good to go!")

In [ ]:
# @title <font size="5">↓ ឵឵<i>Import specific requirements</font> { vertical-output: true }
import os
import sys

try:
    if "kso_utils" not in sys.modules:
        sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), ".."))) 
        import kso_utils.kso_utils

        sys.modules["kso_utils"] = kso_utils.kso_utils
        print("Using development version...")
        # Enables testing changes in utils
        %load_ext autoreload
        %autoreload 2
except:
    print("Installing latest version from PyPI...")
    %pip install -q kso-utils

# Enables testing changes in utils
%load_ext autoreload
%autoreload 2

# Set to display dataframes as interactive tables
from itables import init_notebook_mode

init_notebook_mode(all_interactive=True)

# Import required modules
import kso_utils.tutorials_utils as t_utils
import kso_utils.project_utils as p_utils
import kso_utils.t6_utils as t6
from kso_utils.project import ProjectProcessor, MLProjectProcessor

clear_output()
print("Packages loaded successfully")

In [ ]:
# @title <font size="5">↓ ឵឵<i>Choose your project</font> { vertical-output: true }
project_name = t_utils.choose_project()

In [ ]:
project = p_utils.find_project(project_name=project_name.value)

In [ ]:
# Initialise pp
pp = ProjectProcessor(project)

In [ ]:
# Initialise mlp
mlp = MLProjectProcessor(pp)

# Evaluate model on custom footage

In [ ]:
# @title <font size="5">↓ ឵឵<i>Choose the model</font> { vertical-output: true }
model = t6.choose_model(project_name.value, team_name=mlp.team_name)

In [ ]:
# @title <font size="5">↓ ឵឵<i>Choose folder to download the model</font> { vertical-output: true }
download_dir = t_utils.choose_folder(".", "where to download the model")

In [ ]:
# @title <font size="5">↓ ឵឵឵឵<i>Download model</font> { vertical-output: true }
artifact_dir = mlp.get_model(model.value, download_dir.selected)

In [ ]:
# @title <font size="5">↓ ឵឵឵឵<i>Choose your custom footage</font> { vertical-output: true }
source = t_utils.choose_footage(
    project,
    project.movie_folder if project.movie_folder not in [None, "None"] else ".",
    "custom footage",
)

In [ ]:
source_value = t_utils.process_source(source)

In [ ]:
# @title <font size="5">↓ ឵឵<i>Choose folder where to save the runs</font> { vertical-output: true }
# This should be left as default value in most cases.
save_dir = t_utils.choose_folder(".", "runs output")

In [ ]:
# @title <font size="5">↓ ឵឵<i>Choose a confidence threshold for evaluation</font> { vertical-output: true }
conf_thres = t6.choose_conf()

In [ ]:
# @title <font size="5">↓ ឵឵<i>Run model over selected custom footage</font> { vertical-output: true }
mlp.detect_yolov5(
    source=source_value,
    save_dir=save_dir.selected,
    conf_thres=conf_thres.value,
    artifact_dir=artifact_dir,
)

In [ ]:
# @title <font size="5">↓ ឵឵<i>Choose folder with the evaluation data</font> { vertical-output: true }
eval_dir = t_utils.choose_folder(
    save_dir.selected
    if "save_dir" in vars() and save_dir.selected is not None
    else ".",
    "runs output",
)

In [ ]:
# @title <font size="5">↓ ឵឵<i>Add the data to Weights and Biases</font> { vertical-output: true }
mlp.save_detections_wandb(conf_thres.value, model.value, eval_dir.selected)

In [ ]:
# @title <font size="5">↓ ឵឵<i>View model output</font> { vertical-output: true }
viewer = t6.choose_files(eval_dir.selected)

In [ ]:
# @title <font size="5">↓ ឵឵<i>Investigate training and validation datasets (only image data)</font> { vertical-output: true }
train_dataset, val_dataset = t6.get_dataset(project_name.value, model.value)

In [ ]:
# Training set
t6.get_data_viewer(os.path.join(train_dataset, "data/images"))

In [ ]:
# Validation set
t6.get_data_viewer(os.path.join(val_dataset, "data/images"))

# Track unique individuals (Optional)

In [ ]:
mlp.track_individuals(
    source=source_value,
    artifact_dir=artifact_dir,
    eval_dir=eval_dir.selected,
    conf_thres=conf_thres.value,
    img_size=(540, 540),
)

In [ ]:
# END